# high level goal- figure out the direction which is encoding for the backup

# Imports

In [ ]:
%pip install git+https://github.com/neelnanda-io/TransformerLens.git
%pip install git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python
%pip install git+https://github.com/neelnanda-io/neel-plotly.git
import os
import sys
if not os.path.exists("path_patching.py"):
        !wget https://github.com/callummcdougall/path_patching/archive/refs/heads/main.zip
        !unzip /content/main.zip 'path_patching-main/ioi_dataset.py'
        !unzip /content/main.zip 'path_patching-main/path_patching.py'
        sys.path.append("/content/path_patching-main")
        os.remove("/content/main.zip")
        os.rename("/content/path_patching-main/ioi_dataset.py", "ioi_dataset.py")
        os.rename("/content/path_patching-main/path_patching.py", "path_patching.py")
        os.rmdir("/content/path_patching-main")

from path_patching import Node, IterNode, path_patch, act_patch

  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-5i1d1ugf
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-5i1d1ugf
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 65b5a6ff132e14c92db34523083a18698b960e76
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

  Cloning https://github.com/callummcdougall/CircuitsVis.git to /tmp/pip-req-build-rzak0_8q
  Running command git clone --filter=blob:none --quiet https://github.com/callummcdougall/CircuitsVis.git /tmp/pip-req-build-rzak0_8q
  Resolved https://github.com/callummcdougall/CircuitsVis.git to commit 1ec4a8f8fa4368e95500fe3a188b367500af5f98
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for circuitsvis: filename=circuitsvis-0.0.0-py3-none-any.whl size=1808606 sha256=dd4eaa3398f1b2f22b1abf97ed61d90d1a6923648746118d11f33bd373b04414
  Stored in directory: /tmp/pip-ephem-wheel-cache-4blg3kxa/wheels/86/be/ad/78078aba9344d200aad61b63d35cdaecdec160212f039eed74
Successfully built circuitsvis
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4
  Cloni

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor

import numpy as np
import pandas as pd
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import random
from pathlib import Path
# import plotly.express as px
from torch.utils.data import DataLoader
from typing import Union, List, Optional, Callable, Tuple, Dict, Literal, Set
from jaxtyping import Float, Int
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.utils import to_numpy
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache, patching

import plotly.express as px
import circuitsvis as cv
import os, sys

In [ ]:
if not os.path.exists("path_patching.py"):
        !wget https://github.com/callummcdougall/path_patching/archive/refs/heads/main.zip
        !unzip /content/main.zip 'path_patching-main/ioi_dataset.py'
        !unzip /content/main.zip 'path_patching-main/path_patching.py'
        sys.path.append("/content/path_patching-main")
        os.remove("/content/main.zip")
        os.rename("/content/path_patching-main/ioi_dataset.py", "ioi_dataset.py")
        os.rename("/content/path_patching-main/path_patching.py", "path_patching.py")
        os.rmdir("/content/path_patching-main")

from path_patching import Node, IterNode, path_patch, act_patch

In [ ]:
#%pip install git+https://github.com/neelnanda-io/neel-plotly.git
from neel_plotly import imshow, line, scatter, histogram
import tqdm
torch.set_grad_enabled(False)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
device

'cuda'

In [ ]:
if not os.path.exists("path_patching.py"):
        !wget https://github.com/callummcdougall/path_patching/archive/refs/heads/main.zip
        !unzip main.zip 'path_patching-main/ioi_dataset.py'
        !unzip main.zip 'path_patching-main/path_patching.py'
        sys.path.append("/path_patching-main")
        os.remove("main.zip")
        os.rename("/path_patching-main/ioi_dataset.py", "ioi_dataset.py")
        os.rename("/path_patching-main/path_patching.py", "path_patching.py")
        os.rmdir("/path_patching-main")

from path_patching import Node, IterNode, path_patch, act_patch

In [ ]:
update_layout_set = {
    "xaxis_range", "yaxis_range", "hovermode", "xaxis_title", "yaxis_title", "colorbar", "colorscale", "coloraxis", "title_x", "bargap", "bargroupgap", "xaxis_tickformat",
    "yaxis_tickformat", "title_y", "legend_title_text", "xaxis_showgrid", "xaxis_gridwidth", "xaxis_gridcolor", "yaxis_showgrid", "yaxis_gridwidth", "yaxis_gridcolor",
    "showlegend", "xaxis_tickmode", "yaxis_tickmode", "xaxis_tickangle", "yaxis_tickangle", "margin", "xaxis_visible", "yaxis_visible", "bargap", "bargroupgap"
}

def imshow(tensor, return_fig = False, renderer=None, **kwargs):
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    facet_labels = kwargs_pre.pop("facet_labels", None)
    border = kwargs_pre.pop("border", False)
    if "color_continuous_scale" not in kwargs_pre:
        kwargs_pre["color_continuous_scale"] = "RdBu"
    if "margin" in kwargs_post and isinstance(kwargs_post["margin"], int):
        kwargs_post["margin"] = dict.fromkeys(list("tblr"), kwargs_post["margin"])
    if "color_continuous_midpoint" not in kwargs_pre:
        fig = px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, **kwargs_pre)
    else:
        fig = px.imshow(utils.to_numpy(tensor), **kwargs_pre)
    if facet_labels:
        for i, label in enumerate(facet_labels):
            fig.layout.annotations[i]['text'] = label
    if border:
        fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
        fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    # things like `xaxis_tickmode` should be applied to all subplots. This is super janky lol but I'm under time pressure
    for setting in ["tickangle"]:
      if f"xaxis_{setting}" in kwargs_post:
          i = 2
          while f"xaxis{i}" in fig["layout"]:
            kwargs_post[f"xaxis{i}_{setting}"] = kwargs_post[f"xaxis_{setting}"]
            i += 1
    fig.update_layout(**kwargs_post)
    fig.show(renderer=renderer)
    if return_fig:
      return fig

def hist(tensor, renderer=None, **kwargs):
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    names = kwargs_pre.pop("names", None)
    if "barmode" not in kwargs_post:
        kwargs_post["barmode"] = "overlay"
    if "bargap" not in kwargs_post:
        kwargs_post["bargap"] = 0.0
    if "margin" in kwargs_post and isinstance(kwargs_post["margin"], int):
        kwargs_post["margin"] = dict.fromkeys(list("tblr"), kwargs_post["margin"])
    fig = px.histogram(x=tensor, **kwargs_pre).update_layout(**kwargs_post)
    if names is not None:
        for i in range(len(fig.data)):
            fig.data[i]["name"] = names[i // 2]
    fig.show(renderer)

In [ ]:
from plotly import graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,

    device = device
)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


# Model + Dataset Setup

## lists

In [ ]:
objects = [
  "perfume",
  "scissors",
  "drum",
  "trumpet",
  "phone",
  "football",
  "token",
  "bracelet",
  "badge",
  "novel",
  "pillow",
  "coffee",
  "skirt",
  "balloon",
  "photo",
  "plate",
  "headphones",
  "flask",
  "menu",
  "compass",
  "belt",
  "wallet",
  "pen",
  "mask",
  "ticket",
  "suitcase",
  "sunscreen",
  "letter",
  "torch",
  "cocktail",
  "spoon",
  "comb",
  "shirt",
  "coin",
  "cable",
  "button",
  "recorder",
  "frame",
  "key",
  "card",
  "canvas",
  "packet",
  "bowl",
  "receipt",
  "pan",
  "report",
  "book",
  "cap",
  "charger",
  "rake",
  "fork",
  "map",
  "soap",
  "cash",
  "whistle",
  "rope",
  "violin",
  "scale",
  "diary",
  "ruler",
  "mouse",
  "toy",
  "cd",
  "dress",
  "shampoo",
  "flashlight",
  "newspaper",
  "puzzle",
  "tripod",
  "brush",
  "cane",
  "whisk",
  "tablet",
  "purse",
  "paper",
  "vinyl",
  "camera",
  "guitar",
  "necklace",
  "mirror",
  "cup",
  "cloth",
  "flag",
  "socks",
  "shovel",
  "cooler",
  "hammer",
  "shoes",
  "chalk",
  "wrench",
  "towel",
  "glove",
  "speaker",
  "remote",
  "leash",
  "magazine",
  "notebook",
  "candle",
  "feather",
  "gloves",
  "mascara",
  "charcoal",
  "pills",
  "laptop",
  "pamphlet",
  "knife",
  "kettle",
  "scarf",
  "tie",
  "goggles",
  "fins",
  "lipstick",
  "shorts",
  "joystick",
  "bookmark",
  "microphone",
  "hat",
  "pants",
  "umbrella",
  "harness",
  "roller",
  "blanket",
  "folder",
  "bag",
  "crate",
  "pot",
  "watch",
  "mug",
  "sandwich",
  "yarn",
  "ring",
  "backpack",
  "glasses",
  "pencil",
  "broom",
  "baseball",
  "basket",
  "loaf",
  "coins",
  "bakery",
  "tape",
  "helmet",
  "bible",
  "jacket"
]

In [ ]:
names = [
  " Sebastian",
  " Jack",
  " Jeremiah",
  " Ellie",
  " Sean",
  " William",
  " Caroline",
  " Cooper",
  " Xavier",
  " Ian",
  " Mark",
  " Brian",
  " Carter",
  " Nicholas",
  " Peyton",
  " Luke",
  " Alexis",
  " Ted",
  " Jan",
  " Ty",
  " Jen",
  " Sophie",
  " Kelly",
  " Claire",
  " Leo",
  " Nolan",
  " Kyle",
  " Ashley",
  " Samantha",
  " Avery",
  " Jackson",
  " Hudson",
  " Rebecca",
  " Robert",
  " Joshua",
  " Olivia",
  " Reagan",
  " Lauren",
  " Chris",
  " Chelsea",
  " Deb",
  " Chloe",
  " Madison",
  " Kent",
  " Thomas",
  " Oliver",
  " Dylan",
  " Ann",
  " Audrey",
  " Greg",
  " Henry",
  " Emma",
  " Josh",
  " Mary",
  " Daniel",
  " Carl",
  " Scarlett",
  " Ethan",
  " Levi",
  " Eli",
  " James",
  " Patrick",
  " Isaac",
  " Brooke",
  " Alexa",
  " Eleanor",
  " Anthony",
  " Logan",
  " Damian",
  " Jordan",
  " Tyler",
  " Haley",
  " Isabel",
  " Alan",
  " Lucas",
  " Dave",
  " Susan",
  " Joseph",
  " Brad",
  " Joe",
  " Vincent",
  " Maya",
  " Will",
  " Jessica",
  " Sophia",
  " Angel",
  " Steve",
  " Benjamin",
  " Eric",
  " Cole",
  " Justin",
  " Amy",
  " Nora",
  " Seth",
  " Anna",
  " Stella",
  " Frank",
  " Larry",
  " Alexandra",
  " Ken",
  " Lucy",
  " Katherine",
  " Leah",
  " Adrian",
  " David",
  " Liam",
  " Christian",
  " John",
  " Nathaniel",
  " Andrea",
  " Laura",
  " Kim",
  " Kevin",
  " Colin",
  " Marcus",
  " Emily",
  " Sarah",
  " Steven",
  " Eva",
  " Richard",
  " Faith",
  " Amelia",
  " Harper",
  " Keith",
  " Ross",
  " Megan",
  " Brooklyn",
  " Tom",
  " Grant",
  " Savannah",
  " Riley",
  " Julia",
  " Piper",
  " Wyatt",
  " Jake",
  " Nathan",
  " Nick",
  " Blake",
  " Ryan",
  " Jason",
  " Chase",]

In [ ]:
# names = [

#     " Mary", " John",
#     " Tom", " James",
#     " Dan", " Sid"  ,
#     " Martin", " Amy",
#     " Cody", " Jay",
#     " Jack", " Jill",
#     " Mark", " Martin",
#     " Sarah", " Emily",
#     " Cole", " George",
#     " Kai", " Bryce",
# ]

In [ ]:
places = [
  "swamp",
  "school",
  "volcano",
  "hotel",
  "subway",
  "arcade",
  "library",
  "island",
  "convent",
  "pool",
  "mall",
  "prison",
  "quarry",
  "temple",
  "ruins",
  "factory",
  "zoo",
  "mansion",
  "tavern",
  "planet",
  "forest",
  "airport",
  "pharmacy",
  "church",
  "park",
  "delta",
  "mosque",
  "valley",
  "casino",
  "pyramid",
  "aquarium",
  "castle",
  "ranch",
  "clinic",
  "theater",
  "gym",
  "studio",
  "station",
  "palace",
  "stadium",
  "museum",
  "plateau",
  "home",
  "resort",
  "garage",
  "reef",
  "lounge",
  "chapel",
  "canyon",
  "brewery",
  "market",
  "jungle",
  "office",
  "cottage",
  "street",
  "gallery",
  "landfill",
  "glacier",
  "barracks",
  "bakery",
  "synagogue",
  "jersey",
  "plaza",
  "garden",
  "cafe",
  "cinema",
  "beach",
  "harbor",
  "circus",
  "bridge",
  "monastery",
  "desert",
  "tunnel",
  "motel",
  "fortress"
]

## code


In [ ]:
template = "When{name_A} and{name_B} went to the {place},{name_C} gave the {object} to"

In [ ]:
# prompt: generate a list of prompts using the template. ensure that no prompt uses the same two names

import random

# Create a list of all possible pairs of names
names_list = list(itertools.combinations(names, 2))

# Create a list of prompts
prompts = []
counter_prompts = []


name_pairs = []
corrupt_name_triplets = []
#counter_name_pairs = []

for name_pair in names_list:
    name_A, name_B = name_pair
    # Generate a random place
    place = random.choice(places)
    # Generate a random object
    objectA = random.choice(objects)
    # Create a prompt
    prompt = template.format(
        name_A=name_A,
        name_B=name_B,
        place=place,
        name_C=name_A,
        object=objectA,
    )
    prompts.append(prompt)
    name_pairs.append([name_B, name_A])


    # generate three other names that are not name_A and name_B
    other_names = []
    while(len(other_names) != 3):
      new_name = random.choice(names)
      if new_name is not name_A and new_name is not name_B and new_name not in other_names:
        other_names.append(new_name)

    counter_prompts.append(template.format(

        name_A=other_names[0],
        name_B=other_names[1],
        place=place,
        name_C=other_names[2],
        object=objectA,

    ))

    corrupt_name_triplets.append([other_names[0], other_names[1], other_names[2]])
# Print the prompts

In [ ]:
# generate random list of numbers
batch_size = 20
rand_indices = torch.randint(0, len(prompts), size = (batch_size,))

# indices plus the indices + 1
#double_rand_indices = torch.cat((rand_indices, rand_indices + 1))

In [ ]:
rand_indices

tensor([2998, 8423, 2065, 5197, 8683, 9654, 1706, 1146, 5185, 5590,  400, 6112,
        2467, 8998, 4125, 2069, 9324,   86, 8326, 3765])

In [ ]:
clean_prompts = [prompts[i] for i in rand_indices]
corrupted_prompts = [counter_prompts[i] for i in rand_indices]
name_answers = [name_pairs[i] for i in rand_indices]
corrupt_name_answer_triplets = [corrupt_name_triplets[i] for i in rand_indices]

In [ ]:
clean_tokens = model.to_tokens(clean_prompts, prepend_bos = True).cuda()
corrupted_tokens = model.to_tokens(corrupted_prompts, prepend_bos = True).cuda()
answer_tokens = torch.concat([
    model.to_tokens(names, prepend_bos=False).squeeze(dim=1).unsqueeze(dim=0) for names in name_answers
]).cuda()
corrupt_answer_tokens = torch.concat([
    model.to_tokens(names, prepend_bos=False).squeeze(dim=1).unsqueeze(dim=0) for names in corrupt_name_answer_triplets
]).cuda()


In [ ]:
seq_len = clean_tokens.shape[1]
clean_tokens.shape

torch.Size([20, 15])

In [ ]:
index = 1
clean_prompts[index], corrupted_prompts[index], name_answers[index]

('When Steve and Megan went to the gym, Steve gave the cooler to',
 'When Claire and Amelia went to the gym, Lucas gave the cooler to',
 [' Megan', ' Steve'])

In [ ]:
model.reset_hooks()
clean_logits, clean_cache = model.run_with_cache(clean_tokens, prepend_bos = False)
corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens, prepend_bos = False)

# chillin


In [ ]:
def logits_to_ave_logit_diff(
    logits: Float[Tensor, "batch seq d_vocab"],
    answer_tokens: Float[Tensor, "batch 2"] = answer_tokens,
    per_prompt: bool = False
):
    '''
    Returns logit difference between the correct and incorrect answer.

    If per_prompt=True, return the array of differences rather than the average.
    '''
    # Only the final logits are relevant for the answer
    final_logits: Float[Tensor, "batch d_vocab"] = logits[:, -1, :]
    # Get the logits corresponding to the indirect object / subject tokens respectively
    answer_logits: Float[Tensor, "batch 2"] = final_logits.gather(dim=-1, index=answer_tokens)
    # Find logit difference
    correct_logits, incorrect_logits = answer_logits.unbind(dim=-1)
    answer_logit_diff = correct_logits - incorrect_logits
    return answer_logit_diff if per_prompt else answer_logit_diff.mean()

In [ ]:
clean_per_prompt_diff = logits_to_ave_logit_diff(clean_logits, per_prompt = True)

clean_average_logit_diff = logits_to_ave_logit_diff(clean_logits)
corrupted_average_logit_diff = logits_to_ave_logit_diff(corrupted_logits)

print(clean_average_logit_diff)
print(corrupted_average_logit_diff)

tensor(3.9684, device='cuda:0')
tensor(0.2589, device='cuda:0')


In [ ]:
answer_residual_directions: Float[Tensor, "batch 2 d_model"] = model.tokens_to_residual_directions(answer_tokens)
correct_residual_direction, incorrect_residual_direction = answer_residual_directions.unbind(dim=1)
logit_diff_directions: Float[Tensor, "batch d_model"] = correct_residual_direction - incorrect_residual_direction

In [ ]:
def residual_stack_to_logit_diff(
    residual_stack: Float[Tensor, "... batch d_model"],
    cache: ActivationCache,
    logit_diff_directions: Float[Tensor, "batch d_model"] = logit_diff_directions,
) -> Float[Tensor, "..."]:
    '''
    Gets the avg logit difference between the correct and incorrect answer for a given
    stack of components in the residual stream.
    '''
    batch_size = residual_stack.size(-2)
    scaled_residual_stack = cache.apply_ln_to_stack(residual_stack, layer=-1, pos_slice=-1)



    # # some extra code for more sanity checking
    # new_logits = scaled_residual_stack @ model.W_U
    # print(new_logits.shape)
    # new_logits = einops.repeat(new_logits, "batch d_vocab -> batch 1 d_vocab")
    # print(new_logits.shape)
    # print(logits_to_ave_logit_diff(new_logits))

    return einops.einsum(
        scaled_residual_stack, logit_diff_directions,
        "... batch d_model, batch d_model -> ..."
    ) / batch_size


In [ ]:
answer_residual_directions: Float[Tensor, "batch 2 d_model"] = model.tokens_to_residual_directions(answer_tokens)
print("Answer residual directions shape:", answer_residual_directions.shape)

correct_residual_directions, incorrect_residual_directions = answer_residual_directions.unbind(dim=1)
logit_diff_directions: Float[Tensor, "batch d_model"] = correct_residual_directions - incorrect_residual_directions
print(f"Logit difference directions shape:", logit_diff_directions.shape)

Answer residual directions shape: torch.Size([20, 2, 768])
Logit difference directions shape: torch.Size([20, 768])


In [ ]:
diff_from_unembedding_bias = model.b_U[answer_tokens[:, 0]] -  model.b_U[answer_tokens[:, 1]]

In [ ]:
final_residual_stream: Float[Tensor, "batch seq d_model"] = clean_cache["resid_post", -1]
print(f"Final residual stream shape: {final_residual_stream.shape}")
final_token_residual_stream: Float[Tensor, "batch d_model"] = final_residual_stream[:, -1, :]

print(f"Calculated average logit diff: {(residual_stack_to_logit_diff(final_token_residual_stream, clean_cache, logit_diff_directions) + diff_from_unembedding_bias.mean(0)):.10f}") # <-- okay b_U exists... and matters
print(f"Original logit difference:     {clean_average_logit_diff:.10f}")

Final residual stream shape: torch.Size([20, 15, 768])
Calculated average logit diff: 3.9684185982
Original logit difference:     3.9684174061


# Helper Functions

## Logit Diffs + Gather Important Heads

In [ ]:
def calc_all_logit_diffs(cache):
  clean_per_head_residual, labels = cache.stack_head_results(layer = -1, return_labels = True, apply_ln = False) # per_head_residual.shape = heads batch seq_pos d_model
  # also, for the worried, no, we're not missing the application of LN here since it gets applied in the below function call
  per_head_logit_diff: Float[Tensor, "batch head"] = residual_stack_to_logit_diff(clean_per_head_residual[:, :, -1, :], cache)

  per_head_logit_diff = einops.rearrange(
      per_head_logit_diff,
      "(layer head) ... -> layer head ...",
      layer=model.cfg.n_layers
  )

  correct_direction_per_head_logit: Float[Tensor, "batch head"] = residual_stack_to_logit_diff(clean_per_head_residual[:, :, -1, :], cache, logit_diff_directions = correct_residual_direction)

  correct_direction_per_head_logit = einops.rearrange(
      correct_direction_per_head_logit,
      "(layer head) ... -> layer head ...",
      layer=model.cfg.n_layers
  )

  incorrect_direction_per_head_logit: Float[Tensor, "batch head"] = residual_stack_to_logit_diff(clean_per_head_residual[:, :, -1, :], cache, logit_diff_directions = incorrect_residual_direction)

  incorrect_direction_per_head_logit = einops.rearrange(
      incorrect_direction_per_head_logit,
      "(layer head) ... -> layer head ...",
      layer=model.cfg.n_layers
  )

  return per_head_logit_diff, correct_direction_per_head_logit, incorrect_direction_per_head_logit

per_head_logit_diff, correct_direction_per_head_logit, incorrect_direction_per_head_logit = calc_all_logit_diffs(clean_cache)

Tried to stack head results when they weren't cached. Computing head results now


In [ ]:
top_heads = []
k = 5

flattened_tensor = per_head_logit_diff.flatten().cpu()
_, topk_indices = torch.topk(flattened_tensor, k)
top_layer_arr, top_index_arr = np.unravel_index(topk_indices.numpy(), per_head_logit_diff.shape)

for l, i in zip(top_layer_arr, top_index_arr):
  top_heads.append((l,i))

print(top_heads)

[(9, 9), (9, 6), (10, 0), (10, 10), (10, 6)]


In [ ]:
per_head_logit_diff[11]

tensor([ 0.0056, -0.1579, -0.4142,  0.0035, -0.0158,  0.0076,  0.0241, -0.0381,
         0.0064, -0.0390, -1.0198, -0.0407], device='cuda:0')

In [ ]:
neg_heads = []
neg_indices = torch.nonzero(torch.lt(per_head_logit_diff, -0.1))
neg_heads_list = neg_indices.squeeze().tolist()
for i in neg_heads_list:
  neg_heads.append((i[0], i[1]))

print(neg_heads)

[(10, 3), (10, 7), (11, 1), (11, 2), (11, 10)]


In [ ]:
def display_all_logits(cache, title = "Logit Contributions", comparison = False, return_fig = False, logits = None):

  a,b,c = calc_all_logit_diffs(cache)
  if logits is not None:
    ld = logits_to_ave_logit_diff(logits)
  else:
    ld = 0.00

  if not comparison:
    fig = imshow(
        torch.stack([a,b,c]),
        return_fig = True,
        facet_col = 0,
        facet_labels = [f"Logit Diff - {ld:.2f}", "Correct Direction", "Incorrect Direction"],
        title=title,
        labels={"x": "Head", "y": "Layer", "color": "Logit Contribution"},
        #coloraxis=dict(colorbar_ticksuffix = "%"),
        border=True,
        width=1500,
        margin={"r": 100, "l": 100}
    )
  else:

    ca, cb, cc = calc_all_logit_diffs(clean_cache)
    fig = imshow(
        torch.stack([a, b, c, a - ca, b - cb, c - cc]),
        return_fig = True,
        facet_col = 0,
        facet_labels = [f"Logit Diff - {ld:.2f}", "Correct Direction", "Incorrect Direction", "Logit Diff Diff", "Correction Direction Diff", "Incorrect Direction Diff"],
        title=title,
        labels={"x": "Head", "y": "Layer", "color": "Logit Contribution"},
        #coloraxis=dict(colorbar_ticksuffix = "%"),
        border=True,
        width=1500,
        margin={"r": 100, "l": 100}
    )


  if return_fig:
    return fig



fig = display_all_logits(clean_cache, title = "Logit Contributions on Clean Dataset", return_fig = True, logits = clean_logits)

In [ ]:
clean_cache

ActivationCache with keys ['hook_embed', 'hook_pos_embed', 'blocks.0.hook_resid_pre', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.hook_attn_out', 'blocks.0.hook_resid_mid', 'blocks.0.ln2.hook_scale', 'blocks.0.ln2.hook_normalized', 'blocks.0.mlp.hook_pre', 'blocks.0.mlp.hook_post', 'blocks.0.hook_mlp_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.ln1.hook_scale', 'blocks.1.ln1.hook_normalized', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook_k', 'blocks.1.attn.hook_v', 'blocks.1.attn.hook_attn_scores', 'blocks.1.attn.hook_pattern', 'blocks.1.attn.hook_z', 'blocks.1.hook_attn_out', 'blocks.1.hook_resid_mid', 'blocks.1.ln2.hook_scale', 'blocks.1.ln2.hook_normalized', 'blocks.1.mlp.hook_pre', 'blocks.1.mlp.hook_post', 'blocks.1.hook_mlp_out', 'blocks.1.hook_resid_post', 'blocks.2.hook_re

In [ ]:
# # testing
# #resid_layer_ten = clean_cache[utils.get_act_name("resid_pre", 10)]
# #new_residual_stream = model.blocks[10].ln1(resid_layer_ten)

# new_residual_stream = clean_cache["blocks.9.ln1.hook_normalized"]
# clean_ten_two_predict = simulate_head(new_residual_stream, 9, 10)[:, -1, :]
# actual_output = get_output_of_head(9, 10, clean_cache)

# print(residual_stack_to_logit_diff(actual_output, clean_cache))
# residual_stack_to_logit_diff(clean_ten_two_predict, clean_cache)

In [ ]:
def stare_at_attention_and_head_pat(cache, layer_to_stare_at, head_to_isolate, display_corrupted_text = False, verbose = True, specific = False, specific_index = 0):
  """
  given a cache from a run, displays the attention patterns of a layer, as well as printing out how much the model
  attends to the S1, S2, and IO token
  """

  tokenized_str_tokens = model.to_str_tokens(corrupted_tokens[0]) if display_corrupted_text else model.to_str_tokens(clean_tokens[0])
  attention_patten = cache["pattern", layer_to_stare_at]
  print(f"Layer {layer_to_stare_at} Head {head_to_isolate} Activation Patterns:")


  if not specific:
    S1 = attention_patten.mean(0)[head_to_isolate][-1][2].item()
    IO = attention_patten.mean(0)[head_to_isolate][-1][4].item()
    S2 = attention_patten.mean(0)[head_to_isolate][-1][10].item()
  else:
    S1 = attention_patten[specific_index, head_to_isolate][-1][2].item()
    IO = attention_patten[specific_index, head_to_isolate][-1][4].item()
    S2 = attention_patten[specific_index, head_to_isolate][-1][10].item()


  print("Attention on S1: " + str(S1))
  print("Attention on IO: " + str(IO))
  print("Attention on S2: " + str(S2))
  print("S1 + IO - S2 = " + str(S1 + IO - S2))
  print("S1 + S2 - IO = " + str(S1 + S2 - IO))
  print("S1 - IO - S2 = " + str(S1 - S2 - IO))


  if verbose:
    display(cv.attention.attention_heads(
      tokens=tokenized_str_tokens,
      attention= attention_patten.mean(0) if not specific else attention_patten[specific_index],
      #attention_head_names=[f"L{layer_to_stare_at}H{i}" for i in range(model.cfg.n_heads)],
    ))
  else:
    print(attention_patten.mean(0).shape)

    display(cv.attention.attention_patterns(
      tokens=tokenized_str_tokens,
      attention=attention_patten.mean(0)if not specific else attention_patten[specific_index],
      attention_head_names=[f"L{layer_to_stare_at} H{i}" for i in range(model.cfg.n_heads)],
    ))

In [ ]:
def display_corrupted_clean_logits(cache, title = "Logit Contributions", comparison = False, return_fig = False, logits = None):

  a,b,c = calc_all_logit_diffs(cache)
  if logits is not None:
    ld = logits_to_ave_logit_diff(logits)
  else:
    ld = 0.00

  if not comparison:
    fig = imshow(
        torch.stack([a]),
        return_fig = True,
        facet_col = 0,
        facet_labels = [f"Logit Diff - {ld:.2f}"],
        title=title,
        labels={"x": "Head", "y": "Layer", "color": "Logit Contribution"},
        #coloraxis=dict(colorbar_ticksuffix = "%"),
        border=True,
        width=1500,
        margin={"r": 100, "l": 100}
    )
  else:

    ca, cb, cc = calc_all_logit_diffs(clean_cache)
    fig = imshow(
        torch.stack([a, ca, a - ca]),
        return_fig = True,
        facet_col = 0,
        facet_labels = [f"Ablated Logit Differences: {ld:.2f}", f"Clean Logit Differences: {clean_average_logit_diff:.2f}", f"Difference between Ablated and Clea: {(ld - clean_average_logit_diff):.2f}",],
        title=title,
        labels={"x": "Head", "y": "Layer", "color": "Logit Contribution"},
        #coloraxis=dict(colorbar_ticksuffix = "%"),
        border=True,
        width=1000,
        margin={"r": 100, "l": 100}
    )


  if return_fig:
    return fig
  else:
    return a - ca

In [ ]:
heads =  [(9,9), (9,6), (10,0)]
model.reset_hooks() # callum library buggy
def return_item(item):
  return item

model.reset_hooks()
patched_logits = act_patch(
    model = model,
    orig_input = clean_tokens,
    new_cache = corrupted_cache,
    patching_nodes = [Node("z", layer = layer, head = head) for layer, head in heads],
    patching_metric = return_item,
    verbose = False,
    apply_metric_to_cache = False
)

model.reset_hooks()
noise_sample_ablating_results = act_patch(
    model = model,
    orig_input = clean_tokens,
    new_cache = corrupted_cache,
    patching_nodes = [Node("z", layer = layer, head = head) for layer, head in heads],
    patching_metric = partial(display_corrupted_clean_logits, title = f"Logit Differences When Sample Ablating in Name Mover Heads", comparison = True, logits = patched_logits),
    verbose = False,
    apply_metric_to_cache = True
)

Tried to stack head results when they weren't cached. Computing head results now


In [ ]:
heads =  [(9,i) for i in range(12)] +  [(10,i) for i in range(12)]
model.reset_hooks() # callum library buggy
def return_item(item):
  return item

model.reset_hooks()
patched_logits = act_patch(
    model = model,
    orig_input = clean_tokens,
    new_cache = corrupted_cache,
    patching_nodes = [Node("z", layer = layer, head = head) for layer, head in heads],
    patching_metric = return_item,
    verbose = False,
    apply_metric_to_cache = False
)

model.reset_hooks()
all_layernine_noise_patching_results = act_patch(
    model = model,
    orig_input = clean_tokens,
    new_cache = corrupted_cache,
    patching_nodes = [Node("z", layer = layer, head = head) for layer, head in heads],
    patching_metric = partial(display_corrupted_clean_logits, title = f"Logits When Sample Ablating in Layer 9", comparison = True, logits = patched_logits),
    verbose = False,
    apply_metric_to_cache = True
)

Tried to stack head results when they weren't cached. Computing head results now


In [ ]:
heads =  [(9,9)]
model.reset_hooks() # callum library buggy
def return_item(item):
  return item

model.reset_hooks()
patched_logits = act_patch(
    model = model,
    orig_input = clean_tokens,
    new_cache = corrupted_cache,
    patching_nodes = [Node("z", layer = layer, head = head) for layer, head in heads],
    patching_metric = return_item,
    verbose = False,
    apply_metric_to_cache = False
)

nine_nine_sample_ablating_cache = act_patch(
    model = model,
    orig_input = clean_tokens,
    new_cache = corrupted_cache,
    patching_nodes = [Node("z", layer = layer, head = head) for layer, head in heads],
    patching_metric = return_item,
    verbose = False,
    apply_metric_to_cache = True
)


model.reset_hooks()
nine_nine_sample_ablating_results = act_patch(
    model = model,
    orig_input = clean_tokens,
    new_cache = corrupted_cache,
    patching_nodes = [Node("z", layer = layer, head = head) for layer, head in heads],
    patching_metric = partial(display_corrupted_clean_logits, title = f"Logit Differences When Sample Ablating in Name Mover Heads", comparison = True, logits = patched_logits),
    verbose = False,
    apply_metric_to_cache = True
)

Tried to stack head results when they weren't cached. Computing head results now


In [ ]:
neg_m_heads = [(10,7), (11,10)]
name_mover_heads = [(9,9), (9,6), (10,0)]
backup_heads = [(9,0), (9,7), (10,1), (10,2), (10,6), (10,10), (11,2), (11,9)]
key_backup_heads = [(10,2), (10,6), (10,10), (11,2)]
strong_neg_backup_heads = [(11,2), (10,2), (10,0), (11,6)]



head_names = ["Negative", "Name Mover", "Backup"]
head_list = [neg_m_heads, name_mover_heads, backup_heads]

In [ ]:
def noising_ioi_metric(
    logits: Float[Tensor, "batch seq d_vocab"],
    clean_logit_diff: float = clean_average_logit_diff,
    corrupted_logit_diff: float = corrupted_average_logit_diff,
) -> float:
    '''
    Given logits, returns how much the performance has been corrupted due to noising.

    We calibrate this so that the value is 0 when performance isn't harmed (i.e. same as IOI dataset),
    and -1 when performance has been destroyed (i.e. is same as ABC dataset).
    '''
    #print(logits[-1, -1])
    patched_logit_diff = logits_to_ave_logit_diff(logits)
    return ((patched_logit_diff - clean_logit_diff) / (clean_logit_diff - corrupted_logit_diff))

print(f"IOI metric (IOI dataset): {noising_ioi_metric(clean_logits):.4f}")
print(f"IOI metric (ABC dataset): {noising_ioi_metric(corrupted_logits):.4f}")

IOI metric (IOI dataset): 0.0000
IOI metric (ABC dataset): -1.0000


In [ ]:
def denoising_ioi_metric(
    logits: Float[Tensor, "batch seq d_vocab"],
    clean_logit_diff: float = clean_average_logit_diff,
    corrupted_logit_diff: float = corrupted_average_logit_diff,
) -> float:
    '''
    We calibrate this so that the value is 1 when performance got restored (i.e. same as IOI dataset),
    and 0 when performance has been destroyed (i.e. is same as ABC dataset).
    '''
    patched_logit_diff = logits_to_ave_logit_diff(logits)
    return ((patched_logit_diff - clean_logit_diff) / (clean_logit_diff - corrupted_logit_diff) + 1)


print(f"IOI metric (IOI dataset): {denoising_ioi_metric(clean_logits):.4f}")
print(f"IOI metric (ABC dataset): {denoising_ioi_metric(corrupted_logits):.4f}")

IOI metric (IOI dataset): 1.0000
IOI metric (ABC dataset): 0.0000


## Query Intervention

In [ ]:
def store_activation(
    activation,
    hook: HookPoint,
    where_to_store
):
    """
    takes a storage container where_to_store, and stores the activation in it at a hook
    """""
    where_to_store[:] = activation

In [ ]:
def kq_rewrite_hook(
    internal_value: Float[Tensor, "batch seq head d_head"],
    hook: HookPoint,
    head,
    unnormalized_resid:  Float[Tensor, "batch seq d_model"],
    vector,
    act_name,
    scale = 1,
    position = -1,
    pre_ln = True
):
  """
  replaces keys or queries with a new result which we get from adding a vector to a position at the residual stream
  head: tuple for head to rewrite keys for
  unnormalized_resid: stored unnormalized residual stream needed to recalculated activations
  """

  ln1 = model.blocks[hook.layer()].ln1
  temp_resid = unnormalized_resid.clone()

  if pre_ln:
    temp_resid[:, position, :] = temp_resid[:, position, :] + scale * vector
    normalized_resid = ln1(temp_resid)
  else:
    temp_resid = ln1(temp_resid)
    temp_resid[:, position, :] = temp_resid[:, position, :] + scale * vector
    normalized_resid = temp_resid


  assert act_name == "q" or act_name == "k"
  if act_name == "q":
    W_Q, b_Q = model.W_Q[head[0], head[1]], model.b_Q[head[0], head[1]]
    internal_value[..., head[1], :] = einops.einsum(normalized_resid, W_Q, "batch seq d_model, d_model d_head -> batch seq d_head") + b_Q

  elif act_name == "k":
    W_K, b_K = model.W_K[head[0], head[1]], model.b_K[head[0], head[1]]
    internal_value[..., head[1], :] = einops.einsum(normalized_resid, W_K, "batch seq d_model, d_model d_head -> batch seq d_head") + b_K


In [ ]:
def patch_head_vector(
    head_vector: Float[Tensor, "batch pos head_index d_head"],
    hook: HookPoint,
    head_indices: int,
    other_cache: ActivationCache
) -> Float[Tensor, "batch pos head_index d_head"]:
    '''
    Patches the output of a given head (before it's added to the residual stream) at
    every sequence position, using the value from the other cache.
    '''
    for head_index in head_indices:
      head_vector[:, :, head_index] = other_cache[hook.name][:, :, head_index]
    return head_vector

In [ ]:
def patch_ln_scale(ln_scale, hook):
  #print(torch.equal(ln_scale, clean_cache["blocks." + str(hook.layer()) + ".ln1.hook_scale"]))
  #print("freezing ln1")
  ln_scale = clean_cache["blocks." + str(hook.layer()) + ".ln1.hook_scale"]
  return ln_scale


def patch_ln2_scale(ln_scale, hook):
  #print(torch.equal(ln_scale, clean_cache["blocks." + str(hook.layer()) + ".ln1.hook_scale"]))
  #print("froze ln2")
  ln_scale = clean_cache["blocks." + str(hook.layer()) + ".ln2.hook_scale"]
  return ln_scale

In [ ]:
def causal_write_into_component(act_comp, head, direction, x, pre_ln = True, result_cache_function = None, result_cache_fun_has_head_input = False, freeze_layernorm = False, ablate_heads = []):
  '''
  writes a vector into the component at a given head
  returns new logit differences of run by default, or pass result_cache_funciton to run on cache

  head - tuple for head to intervene in act_comp for
  direction - vector to add to the act_comp in the head
  x - tensor of amount to scale
  '''
  y = torch.zeros(x.shape)
  for i in range(len(x)):
    scale = x[i]
    model.reset_hooks()
    temp = torch.zeros((batch_size, seq_len, model.cfg.d_model)).cuda()
    model.add_hook(utils.get_act_name("resid_pre", head[0]), partial(store_activation, where_to_store = temp))
    if freeze_layernorm:
      model.add_hook("blocks." + str(head[0]) + ".ln1.hook_scale", patch_ln_scale)
    model.add_hook(utils.get_act_name(act_comp, head[0]), partial(kq_rewrite_hook, head = head, unnormalized_resid = temp, vector = direction, act_name = act_comp, scale = scale, pre_ln = pre_ln))


    if len(ablate_heads) != 0:
      for j in ablate_heads:
        model.add_hook(utils.get_act_name("z", j[0]), partial(patch_head_vector, head_indices = [j[1]], other_cache = corrupted_cache))


    hooked_logits, hooked_cache = model.run_with_cache(clean_tokens)
    model.reset_hooks()


    if result_cache_function != None:
      if not result_cache_fun_has_head_input:
        y[i] = result_cache_function(hooked_cache)
      else:
        y[i] = result_cache_function(hooked_cache, head)
    else:
      # just calculate logit diff
      y[i] = logits_to_ave_logit_diff(hooked_logits)

  return y


In [ ]:
def graph_lines(results, heads, x, title = "Effect of adding/subtracting direction", xtitle = "Scaling on direction", ytitle = "Logit Diff"):
  fig = px.line(title = title)
  for i in range(len(results)):
    fig.add_trace(go.Scatter(x = x, y = results[i], name = str(heads[i])))

  fig.update_xaxes(title = xtitle)
  fig.update_yaxes(title = ytitle)
  fig.show()

In [ ]:
def get_head_IO_minus_S_attn(cache, head, scores = True):

  layer, h_index = head

  if scores:
    attention_patten = cache["attn_scores", layer]
  else:
    attention_patten = cache["pattern", layer]
  S1 = attention_patten.mean(0)[h_index][-1][2].item()
  IO = attention_patten.mean(0)[h_index][-1][4].item()
  S2 = attention_patten.mean(0)[h_index][-1][10].item()

  return IO - S1 - S2


def get_head_IO_minus_just_S1_attn(cache, head, scores = True):

    layer, h_index = head

    if scores:
      attention_patten = cache["attn_scores", layer]
    else:
      attention_patten = cache["pattern", layer]
    S1 = attention_patten.mean(0)[h_index][-1][2].item()
    IO = attention_patten.mean(0)[h_index][-1][4].item()
    S2 = attention_patten.mean(0)[h_index][-1][10].item()

    return IO - S1

def get_head_last_token(cache, head):
  layer, h_index = head
  return cache["pattern", layer][:, h_index, -1, :]


def get_head_attn(cache, head, token, scores = True, mean = True):

  layer, h_index = head

  if scores:
    attention_patten = cache["attn_scores", layer]
  else:
    attention_patten = cache["pattern", layer]


  if mean:
    if token == "S1":
      return attention_patten.mean(0)[h_index][-1][2].item()
    elif token == "IO":
      return attention_patten.mean(0)[h_index][-1][4].item()
    elif token == "S2":
      return attention_patten.mean(0)[h_index][-1][10].item()
    elif token == "BOS":
      return attention_patten.mean(0)[h_index][-1][0].item()
    else:
      print("RAHHHHH YOU MISSTYPED SOMETHING")

  else:
    if token == "S1":
      return attention_patten[:, h_index, -1, 2]
    elif token == "IO":
      return attention_patten[:, h_index, -1, 4]
    elif token == "S2":
      return attention_patten[:, h_index, -1, 10]
    elif token == "BOS":
      return attention_patten[:, h_index, -1, 0]
    else:
      print("RAHHHHH YOU MISSTYPED SOMETHING")


def patch_head_vector(
    head_vector: Float[Tensor, "batch pos head_index d_head"],
    hook: HookPoint,
    head_indices: int,
    other_cache: ActivationCache
) -> Float[Tensor, "batch pos head_index d_head"]:
    '''
    Patches the output of a given head (before it's added to the residual stream) at
    every sequence position, using the value from the other cache.
    '''
    for head_index in head_indices:
      head_vector[:, :, head_index] = other_cache[hook.name][:, :, head_index]
    return head_vector

def get_attn_results_into_head_dirs(heads, direction, scale_amounts, ablate_heads = [], freeze_ln = False, only_S1 = False):
  io_attn_postln_nmh_results = []
  for i in range(len(heads)):
    io_attn_postln_nmh_results.append(causal_write_into_component("q", heads[i], direction, scale_amounts,
                                                        pre_ln = True, freeze_layernorm = freeze_ln, result_cache_function = partial(get_head_attn, token = "IO"), result_cache_fun_has_head_input = True, ablate_heads=ablate_heads))


  s1_attn_postln_nmh_results = []
  for i in range(len(heads)):
    s1_attn_postln_nmh_results.append(causal_write_into_component("q", heads[i], direction, scale_amounts,
                                                        pre_ln = True, freeze_layernorm = freeze_ln,result_cache_function = partial(get_head_attn, token = "S1"), result_cache_fun_has_head_input = True, ablate_heads=ablate_heads))

  s2_attn_postln_nmh_results = []
  for i in range(len(heads)):
    s2_attn_postln_nmh_results.append(causal_write_into_component("q", heads[i], direction, scale_amounts,
                                                        pre_ln = True, freeze_layernorm = freeze_ln,result_cache_function = partial(get_head_attn, token = "S2"), result_cache_fun_has_head_input = True, ablate_heads=ablate_heads))

  diff_results = []
  if not only_S1:
    for i in range(len(heads)):
      diff_results.append(causal_write_into_component("q", heads[i], direction, scale_amounts,
                                                          pre_ln = True, freeze_layernorm = freeze_ln,result_cache_function = get_head_IO_minus_S_attn, result_cache_fun_has_head_input = True, ablate_heads=ablate_heads))
  else:
    for i in range(len(heads)):
      diff_results.append(causal_write_into_component("q", heads[i], direction, scale_amounts,
                                                          pre_ln = True, freeze_layernorm = freeze_ln,result_cache_function = get_head_IO_minus_just_S1_attn, result_cache_fun_has_head_input = True, ablate_heads=ablate_heads))


  bos_attn_postln_nmh_results = []
  for i in range(len(heads)):
    bos_attn_postln_nmh_results.append(causal_write_into_component("q", heads[i], direction, scale_amounts,
                                                        pre_ln = True, freeze_layernorm = freeze_ln,result_cache_function = partial(get_head_attn, token = "BOS"), result_cache_fun_has_head_input = True, ablate_heads=ablate_heads))

  return [io_attn_postln_nmh_results, s1_attn_postln_nmh_results, s2_attn_postln_nmh_results, diff_results, bos_attn_postln_nmh_results]

# Unembedding to Not Ratios

In [ ]:
model.set_use_attn_result(True)

In [ ]:
def get_projection(from_vector, to_vector):
    dot_product = einops.einsum(from_vector, to_vector, "batch d_model, batch d_model -> batch")
    #print("Average Dot Product of Output Across Batch: " + str(dot_product.mean(0)))
    length_of_from_vector = einops.einsum(from_vector, from_vector, "batch d_model, batch d_model -> batch")
    length_of_vector = einops.einsum(to_vector, to_vector, "batch d_model, batch d_model -> batch")




    projected_lengths = (dot_product) / (length_of_vector)
    #print( einops.repeat(projected_lengths, "batch -> batch d_model", d_model = model.cfg.d_model)[0])
    projections = to_vector * einops.repeat(projected_lengths, "batch -> batch d_model", d_model = to_vector.shape[-1])
    return projections

In [ ]:
import torch.nn.functional as F

def compute_cosine_similarity(tensor1, tensor2):
    # Compute cosine similarity
    similarity = F.cosine_similarity(tensor1, tensor2, dim=1)
    return similarity

In [ ]:
def project_vector_operation(
    original_resid_stream: Float[Tensor, "batch seq head_idx d_model"],
    hook: HookPoint,
    vector: Float[Tensor, "batch d_model"],
    position = -1,
    heads = [], # array of ints
    scale_proj = 1,
    project_only = False
) -> Float[Tensor, "batch n_head pos pos"]:
  '''
  Function which gets orthogonal projection of residual stream to a vector, and either subtracts it or keeps only it
  '''
  print("RUH")
  for head in heads:
    projections = get_projection(original_resid_stream[:, position, head, :], vector)
    if project_only:
      original_resid_stream[:, position, head, :] = projections * scale_proj
    else:
      original_resid_stream[:, position, head, :] = (original_resid_stream[:, position, head, :] - projections) * scale_proj #torch.zeros(original_resid_stream[:, position, head, :].shape)#

  return original_resid_stream

In [ ]:
# get ldds when intervening and replacing with directions of corrupted runs
def project_away_component_and_replace_with_something_else(
    original_resid_out: Float[Tensor, "batch seq head_idx d_model"],
    hook: HookPoint,
    project_away_vector: Float[Tensor, "batch d_model"],
    replace_vector : Float[Tensor, "batch d_model"],
    position = -1,
    heads = [], # array of ints,
    project_only = False # whether to, instead of projecting away the vector, keep it!
) -> Float[Tensor, "batch n_head pos pos"]:
    '''
    Function which gets removes a specific component (or keeps only it, if project_only = True) of the an output of a head and replaces it with another vector
    '''
    # right now this projects away the IO direction!
    assert project_away_vector.shape == replace_vector.shape and len(project_away_vector.shape) == 2

    for head in heads:

        head_output = original_resid_out[:, position, head, :]
        projections = get_projection(head_output, project_away_vector)

        if project_only:
            resid_without_projection =  projections
        else:
            resid_without_projection = (head_output - projections)

        updated_resid = resid_without_projection + replace_vector
        original_resid_out[:, position, head, :] = updated_resid

    return original_resid_out

In [ ]:
def patch_last_ln(ln_scale, hook):
  ln_scale = clean_cache["ln_final.hook_scale"]
  return ln_scale

In [ ]:
unembed_io_directions = model.tokens_to_residual_directions(answer_tokens[:, 0])
unembed_s_directions = model.tokens_to_residual_directions(answer_tokens[:, 1])
unembed_diff_directions = unembed_io_directions - unembed_s_directions

unembed_N1_directions = model.tokens_to_residual_directions(corrupt_answer_tokens[:, 0])
unembed_N2_directions = model.tokens_to_residual_directions(corrupt_answer_tokens[:, 1])
unembed_N3_directions = model.tokens_to_residual_directions(corrupt_answer_tokens[:, 2])

heads_whose_output_to_replace = [(9,9)] # note that if we do anything abote alyer 9, we may need to add more LNs

target_intervene_direction = unembed_io_directions
ln_on = True
ca, cb, cc = calc_all_logit_diffs(clean_cache)

In [ ]:
def project_stuff_on_heads(project_heads, direction, project_only = False, scale_proj = 1, output = "display_logits", freeze_ln = False):
    model.reset_hooks()

    # project_heads is a list of tuples (layer, head). for each layer, write a hook which projects all the heads from the layer
    for layer in range(model.cfg.n_layers):
        key_heads = [head[1] for head in project_heads if head[0] == layer]
        if len(key_heads) > 0:
            #print(key_heads)
            model.add_hook(utils.get_act_name("result", layer), partial(project_vector_operation, vector = direction, heads = key_heads, scale_proj = scale_proj, project_only = project_only))

    if freeze_ln:
        for layer in [9,10,11]:
            model.add_hook("blocks." + str(layer) + ".ln1.hook_scale", patch_ln_scale)
            model.add_hook("blocks." + str(layer) + ".ln2.hook_scale", patch_ln2_scale)
        model.add_hook("ln_final.hook_scale", patch_last_ln)

    hooked_logits, hooked_cache = model.run_with_cache(clean_tokens)
    model.reset_hooks()
    if output == "display_logits":
        return display_all_logits(hooked_cache, comparison=True, logits = hooked_logits, title = f"Projecting {('only' if project_only else 'away')} IO direction in heads {project_heads}")
    elif output == "get_ldd":
        a,_,_ = calc_all_logit_diffs(hooked_cache)

        return a - ca

In [ ]:
def compare_intervention_ldds_with_sample_ablated(all_ldds, ldds_names, heads = key_backup_heads, just_logits = False):
    results = torch.zeros((len(all_ldds), len(heads)))


    if just_logits:
        for ldd_index, compare_ldds in enumerate(all_ldds):
            for i, head in enumerate(heads):
                #print(head)
                results[ldd_index, i] = ((compare_ldds[head[0], head[1]]).item()) # / noise_sample_ablating_results[head[0], head[1]]).item())
    else:
        for ldd_index, compare_ldds in enumerate(all_ldds):
            for i, head in enumerate(heads):
                #print(head)
                results[ldd_index, i] = ((compare_ldds[head[0], head[1]] / noise_sample_ablating_results[head[0], head[1]]).item())

    return imshow(
        results,
        #facet_col = 0,
        #labels = [f"Head {head}" for head in key_backup_heads],
        title=f"The {'Ratio of Backup (Logit Diff Diff)' if not just_logits else 'Logit Diff Diffs'} of Intervention" + ("to Sample Ablation Backup" if not just_logits else ""),
        labels={"x": "Receiver Head", "y": "Intervention", "color": "Ratio of Logit Diff Diff to Sample Ablation" if not just_logits else "Logit Diff Diff"},
        #coloraxis=dict(colorbar_ticksuffix = "%"),
        # range of y-axis color from 0 to 2
        #color_continuous_scale="mint",
        color_continuous_midpoint=1 if not just_logits else 0,
        # give x-axis labels
        x = [str(head) for head in heads],
        y = ldds_names,
        border=True,
        width=900,
        height = 50 * len(all_ldds) + 300,
        margin={"r": 100, "l": 100},
        # show the values of the results above the heatmap
        text_auto = True,
        return_fig = True
    )

get results from replacing all IO directions

In [ ]:
def run_interventions():
    zero_ablate_all_heads_ldds = project_stuff_on_heads(heads_whose_output_to_replace, target_intervene_direction, project_only = True, scale_proj = 0, output = "get_ldd", freeze_ln=ln_on)
    project_only_io_direction = project_stuff_on_heads(heads_whose_output_to_replace, target_intervene_direction, project_only = True, scale_proj = 1, output = "get_ldd", freeze_ln=ln_on)
    project_away_io_direction = project_stuff_on_heads(heads_whose_output_to_replace, target_intervene_direction, project_only = False, scale_proj = 1, output = "get_ldd", freeze_ln=ln_on)

    target_heads = heads_whose_output_to_replace
    model.reset_hooks()

    if ln_on:
        for layer in [9,10,11]:
            model.add_hook("blocks." + str(layer) + ".ln1.hook_scale", patch_ln_scale)
            model.add_hook("blocks." + str(layer) + ".ln2.hook_scale", patch_ln2_scale)
        model.add_hook("ln_final.hook_scale", patch_last_ln)

    for head in target_heads:

        # get the output of head on CORRUPTED RUN
        W_O_temp = model.W_O[head[0], head[1]]
        layer_z = corrupted_cache[utils.get_act_name("z", head[0])]
        layer_result = einops.einsum(W_O_temp, layer_z, "d_head d_model, batch seq h_idx d_head -> batch seq h_idx d_model")
        output_head = layer_result[:, -1, head[1], :]

        # get projection of CORRUPTED HEAD OUTPUT onto IO token
        corrupted_head_only_IO_output = get_projection(output_head, target_intervene_direction)

        # add hook to now replace with this corrupted IO direction
        model.add_hook(utils.get_act_name("result", head[0]), partial(project_away_component_and_replace_with_something_else, project_away_vector = target_intervene_direction, heads = [head[1]], replace_vector = corrupted_head_only_IO_output))

    replace_with_new_IO_logits, replace_with_new_IO_cache = model.run_with_cache(clean_tokens)

    replace_all_IOs_ldds = calc_all_logit_diffs(replace_with_new_IO_cache)[0] - ca
    model.reset_hooks()

    target_heads = heads_whose_output_to_replace
    model.reset_hooks()
    if ln_on:
        for layer in [9,10,11]:
                    model.add_hook("blocks." + str(layer) + ".ln1.hook_scale", patch_ln_scale)
                    model.add_hook("blocks." + str(layer) + ".ln2.hook_scale", patch_ln2_scale)

    model.add_hook("ln_final.hook_scale", patch_last_ln)
    for head in target_heads:

        # get the output of head on CORRUPTED RUN
        W_O_temp = model.W_O[head[0], head[1]]
        layer_z = corrupted_cache[utils.get_act_name("z", head[0])]
        layer_result = einops.einsum(W_O_temp, layer_z, "d_head d_model, batch seq h_idx d_head -> batch seq h_idx d_model")
        output_head = layer_result[:, -1, head[1], :]


        # get projection of CORRUPTED HEAD OUTPUT onto IO perp token
        corrupted_head_only_IO_output = get_projection(output_head, target_intervene_direction)
        everything_else_but_that = output_head - corrupted_head_only_IO_output

        # add hook to now replace with this corrupted IO perp direction
        model.add_hook(utils.get_act_name("result", head[0]), partial(project_away_component_and_replace_with_something_else, project_away_vector = target_intervene_direction, heads = [head[1]], replace_vector = everything_else_but_that, project_only = True))

    replace_with_new_perp_IO_logits, replace_with_new_perp_IO_cache = model.run_with_cache(clean_tokens)
    replace_all_perp_IOs_ldds = calc_all_logit_diffs(replace_with_new_perp_IO_cache)[0] - ca
    model.reset_hooks()

    return [zero_ablate_all_heads_ldds, project_only_io_direction, replace_all_perp_IOs_ldds, project_away_io_direction, replace_all_IOs_ldds]

In [ ]:
first_intervention = run_interventions()

RUH
RUH
RUH


get results from replacing all perp to IO directions

# and now we arrive at my own personal stuff

In [ ]:
def get_output_of_head(layer, head, cache = clean_cache):
    matrix = model.W_O[layer, head]
    layer_output = cache[utils.get_act_name("z", layer)]
    layer_result = einops.einsum(matrix, layer_output, "d_head d_model, batch seq h_idx d_head -> batch seq h_idx d_model")
    output_of_head = layer_result[:, -1, head, :]
    return output_of_head

In [ ]:
# Find the vector in the direction of both 9.9 and 9.6
nine_six_output = get_output_of_head(9, 6)
nine_nine_output = get_output_of_head(9, 9)
ten_zero_output = get_output_of_head(10, 0)

corrupted_nine_nine_output = get_output_of_head(9, 9, corrupted_cache)

In [ ]:
def get_cleansed_head_ouptut(layer, head):
    output = get_output_of_head(layer, head)
    no_IO = output - get_projection(output, unembed_io_directions)
    no_IO_S = no_IO - get_projection(no_IO, unembed_s_directions)
    return no_IO_S


def get_corrupt_cleansed_head_output(layer, head):
    output = get_output_of_head(layer, head, corrupted_cache)
    no_N1 = output - get_projection(output, unembed_N1_directions)
    no_N1_N2 = no_N1 - get_projection(no_N1, unembed_N2_directions)
    no_N1_N2_N3 = no_N1_N2 - get_projection(no_N1_N2, unembed_N3_directions)
    return no_N1_N2_N3

In [ ]:
cleansed_nine_six = get_cleansed_head_ouptut(9, 6)
cleansed_nine_nine = get_cleansed_head_ouptut(9, 9)

corrupt_cleansed_nine_nine = get_corrupt_cleansed_head_output(9, 9)

nine_six_output_perp_to_IO = nine_six_output - get_projection(nine_six_output, unembed_io_directions)
nine_nine_output_perp_to_IO = nine_nine_output - get_projection(nine_nine_output, unembed_io_directions)

nine_six_IO = get_projection(nine_six_output, unembed_io_directions)
nine_nine_IO = get_projection(nine_nine_output, unembed_io_directions)

diff_between_96_99_no_IO = nine_six_output_perp_to_IO - nine_nine_output_perp_to_IO

In [ ]:

def compute_cosine_similarity(tensor1, tensor2, dim = 1):
    #print(tensor1.shape)
    # Compute cosine similarity
    similarity = F.cosine_similarity(tensor1, tensor2, dim=dim)
    return similarity

In [ ]:
cleansed_nine_nine.shape

torch.Size([20, 768])

In [ ]:
print(compute_cosine_similarity(nine_six_output, nine_nine_output).mean(0))
print(compute_cosine_similarity(nine_six_output_perp_to_IO, nine_nine_output_perp_to_IO).mean(0))
print(compute_cosine_similarity(nine_six_IO, nine_nine_IO).mean(0))
print(compute_cosine_similarity(cleansed_nine_nine, cleansed_nine_six).mean(0))

print("--------- look at 9.9 and its ablated version -------")
print(compute_cosine_similarity(nine_nine_output, corrupted_nine_nine_output).mean(0))
print(compute_cosine_similarity(nine_nine_output_perp_to_IO, corrupted_nine_nine_output).mean(0))
print(compute_cosine_similarity(cleansed_nine_nine, corrupt_cleansed_nine_nine).mean(0))

print("---- and itself? ----")

sum = 0
for index in range(cleansed_nine_nine.shape[0] - 2):
    sum += compute_cosine_similarity(cleansed_nine_nine[index], cleansed_nine_nine[index + 1], dim = 0)

print(sum / (cleansed_nine_nine.shape[0] - 2))

tensor(0.0658, device='cuda:0')
tensor(-0.0643, device='cuda:0')
tensor(1., device='cuda:0')
tensor(-0.0655, device='cuda:0')
--------- look at 9.9 and its ablated version -------
tensor(0.1210, device='cuda:0')
tensor(0.1176, device='cuda:0')
tensor(0.1681, device='cuda:0')
---- and itself? ----
tensor(0.2132, device='cuda:0')


In [ ]:
overlap = get_projection(nine_six_output, nine_nine_output)

# Find overlap with IO
overlap_with_io = get_projection(overlap, unembed_io_directions)
overlap_with_io_perp = overlap - overlap_with_io

In [ ]:
project_stuff_on_heads(heads_whose_output_to_replace, nine_six_output, project_only = True, scale_proj = 1, output = "display_logits", freeze_ln=True)

RUH


In [ ]:
just_9_6 = project_stuff_on_heads( heads_whose_output_to_replace, nine_six_output, project_only = True, scale_proj = 1, output = "get_ldd", freeze_ln=True)
just_9_9 = project_stuff_on_heads(heads_whose_output_to_replace, nine_nine_output, project_only = True, scale_proj = 1, output = "get_ldd", freeze_ln=True)
away_just_9_6 = project_stuff_on_heads(heads_whose_output_to_replace, nine_six_output, project_only = False, scale_proj = 1, output = "get_ldd", freeze_ln=True)
away_just_9_9 = project_stuff_on_heads(heads_whose_output_to_replace, nine_nine_output, project_only = False, scale_proj = 1, output = "get_ldd", freeze_ln=True)



#overlap_9_6_9_9 = project_stuff_on_heads(heads_whose_output_to_replace, overlap, project_only = True, scale_proj = 1, output = "get_ldd", freeze_ln=True)
#away_overlap_9_6_9_9 = project_stuff_on_heads(heads_whose_output_to_replace, overlap, project_only = False, scale_proj = 1, output = "get_ldd", freeze_ln=True)

#ldd_from_96_99_feature_and_parra_to_io = project_stuff_on_heads(heads_whose_output_to_replace, overlap_with_io, project_only = True, scale_proj = 1, output = "get_ldd", freeze_ln=True)
#ldd_from_away_96_99_feature_and_parra_to_io = project_stuff_on_heads(heads_whose_output_to_replace, overlap_with_io, project_only = False, scale_proj = 1, output = "get_ldd", freeze_ln=True)

RUH
RUH
RUH
RUH


In [ ]:
activate_interventions = {
    "Clean Run" : ca - ca,
    #"Sample Ablation of NMHs" : noise_sample_ablating_results,
    "Sample Ablation of (9.9)" : nine_nine_sample_ablating_results,
    "Zero Ablation of NMHs" : first_intervention[0],
    #"Project Only IO Direction (Zero ⊥ IO direction)" : first_intervention[1],
    #"Replace ⊥ IO directions with Corrupted ⊥ IO directions" : first_intervention[2],
    #"Project Away IO Direction (Zero IO direction)" : first_intervention[3],
    #"Replace IO directions with Corrupted IO directions" : first_intervention[4],


    "Project Only (9.6)" : just_9_6,
    "Project Away (9.6)" : away_just_9_6,
    "Project Only (9.9)" : just_9_9,
    "Project Away (9.9)" : away_just_9_9,
    #"Projecting Only (9.6 ∥ 9.9) ∥ IO" : ldd_from_96_99_feature_and_parra_to_io,
    #"Projecting Away (9.6 ∥ 9.9) ∥ IO" : ldd_from_away_96_99_feature_and_parra_to_io,
    #"Project Only (9.6 ∥ 9.9)" : overlap_9_6_9_9,
    #"Project Away (9.6 ∥ 9.9)" : away_overlap_9_6_9_9,

}

In [ ]:
row_labels = [str(i) for i in activate_interventions.keys()]
values = [activate_interventions[key] for key in activate_interventions.keys()]

fig = compare_intervention_ldds_with_sample_ablated(values, row_labels, heads = key_backup_heads , just_logits = True) # + neg_m_heads

In [ ]:
model.set_use_attn_result(False)

# Learn the Backup Vector

Here, we are trying to find the minimal vector which activates Backup. We will attempt this with two constraints:
1) the vector should have low cossim with 9.9 output
2) 9.9 subtracted by this vector should activate maximal backup


1 should optimally converge to zero, and so should 2 (i can do 9.9 Ablation LD - current LD)

## Helper Functions + Metrics

In [ ]:
torch.set_grad_enabled(True)

In [ ]:
model.b_O.shape

torch.Size([12, 768])

In [ ]:

def apply_causal_mask(
        attn_scores: Float[Tensor, "batch n_heads query_pos key_pos"]
    ) -> Float[Tensor, "batch n_heads query_pos key_pos"]:
        '''
        Applies a causal mask to attention scores, and returns masked scores.
        '''
        # Define a mask that is True for all positions we want to set probabilities to zero for
        all_ones = torch.ones(attn_scores.size(-2), attn_scores.size(-1), device=attn_scores.device)
        mask = torch.triu(all_ones, diagonal=1).bool()
        # Apply the mask to attention scores, then return the masked scores
        attn_scores.masked_fill_(mask, 1e-6)
        return attn_scores


def simulate_head(resid, layer, head):
    """
    given a specific head in the model, simulates running a normalized residual stream through it
    note that this does the normal gpt-2 style head. not the fancy stuff.
    """

    # get parameters for head
    W_Q = model.W_Q[layer, head]
    W_K = model.W_K[layer, head]
    W_V = model.W_V[layer, head]
    W_O = model.W_O[layer, head]

    b_Q = model.b_Q[layer, head]
    b_K = model.b_K[layer, head]
    b_V = model.b_V[layer, head]
    #b_O = model.b_O[layer, head]

    # calculate keys, queries, and values
    Q = einops.einsum(resid, W_Q, "batch seq d_model, d_model d_head -> batch seq d_head") + b_Q
    K = einops.einsum(resid, W_K, "batch seq d_model, d_model d_head -> batch seq d_head") + b_K
    V = einops.einsum(resid, W_V, "batch seq d_model, d_model d_head -> batch seq d_head") + b_V

    # calculate attention scores
    attn_scores = einops.einsum(
            Q, K,
            "batch posn_Q d_head, batch posn_K d_head -> batch posn_Q posn_K",
        )

    # mask
    attn_scores = apply_causal_mask(attn_scores/ model.cfg.d_head ** 0.5)

    # calculate attention probs
    attn_probs = F.softmax(attn_scores, dim=-1)

    # weighted sum of values, according to attention probs
    z = einops.einsum(
            V, attn_probs,
            "batch posn_K d_head, batch posn_Q posn_K -> batch posn_Q d_head",
        )

    attn_out = einops.einsum(
            z, W_O,
            "batch posn_Q d_head, d_head d_model -> batch posn_Q d_model")#+ b_O

    return attn_out


In [ ]:
resid_mid_layer_nine = clean_cache[utils.get_act_name("resid_mid", 9)]



sample_ablate_ten_two = get_output_of_head(10, 2, nine_nine_sample_ablating_cache)
sample_ablate_ten_six = get_output_of_head(10, 6, nine_nine_sample_ablating_cache)
sample_ablate_ten_ten = get_output_of_head(10, 10, nine_nine_sample_ablating_cache)

clean_ten_two = get_output_of_head(10, 2)
clean_ten_six = get_output_of_head(10, 6)
clean_ten_ten = get_output_of_head(10, 10)

nine_nine_output = get_output_of_head(9, 9, clean_cache)

In [ ]:
def backup_loss_metric(current_head_output, abs = True, ablated_output = sample_ablate_ten_two):
    # find logit diff of current head output and ablated output
    assert current_head_output.shape == ablated_output.shape
    cur_head_logit_diff = residual_stack_to_logit_diff(current_head_output, nine_nine_sample_ablating_cache)
    ablated_logit_diff = residual_stack_to_logit_diff(ablated_output, nine_nine_sample_ablating_cache)

    #print(cur_head_logit_diff)
    #print(cur_head_logit_diff, ablated_logit_diff)

    if abs:
        return (ablated_logit_diff - cur_head_logit_diff).abs()
    else:
        return ablated_logit_diff - cur_head_logit_diff

def perp_to_9_9_metric(vector, output_9_9 = nine_nine_output):
    expanded_vector = einops.repeat(vector, "d_model -> batch d_model", batch = output_9_9.shape[0])
    return F.cosine_similarity(expanded_vector, output_9_9, dim=-1).mean(0).abs()

def normalize_metric(vector):
    # push down norm of vector
    return einops.einsum(vector, vector, "d_model, d_model -> ") / 100


## Create the Training Loop

In [ ]:
if False:
  learned_vectors = []

In [ ]:
for i in tqdm.tqdm(range(3)):
  trained_vector = torch.randn(model.cfg.d_model, requires_grad=True, device = device)
  optimizer = torch.optim.Adam([trained_vector], lr=0.1)

  for i in range(700):
      optimizer.zero_grad()
      repeated_learned_vector = einops.repeat(trained_vector, "d_model -> batch d_model", batch = nine_nine_output.shape[0])

      # part 1 -- the removing vector should ACTIVATE backup
      new_residual_stream = resid_mid_layer_nine.clone()
      new_residual_stream[:, -1, :] = new_residual_stream[:, -1, :] - get_projection(nine_nine_output, repeated_learned_vector)

      # run through MLP
      new_residual_stream = new_residual_stream + model.blocks[9].mlp(model.blocks[9].ln2(new_residual_stream))
      new_residual_stream = model.blocks[10].ln1(new_residual_stream)

      ten_two_output = simulate_head(new_residual_stream, 10, 2)[:, -1, :]
      ten_six_output = simulate_head(new_residual_stream, 10, 6)[:, -1, :]
      ten_ten_output = simulate_head(new_residual_stream, 10, 10)[:, -1, :]

      activate_loss = backup_loss_metric(ten_ten_output, ablated_output = sample_ablate_ten_ten) + backup_loss_metric(ten_two_output, ablated_output = sample_ablate_ten_two) + backup_loss_metric(ten_six_output, ablated_output = sample_ablate_ten_six)  + perp_to_9_9_metric(trained_vector)



      # part 2 --- if you constrain the heads output to ONLY this, it should NOT activate backup
      second_residual_stream = resid_mid_layer_nine.clone()
      second_residual_stream[:, -1, :] = second_residual_stream[:, -1, :] - nine_nine_output + get_projection(nine_nine_output, repeated_learned_vector)

      # run through MLP
      second_residual_stream = second_residual_stream + model.blocks[9].mlp(model.blocks[9].ln2(second_residual_stream))
      second_residual_stream = model.blocks[10].ln1(second_residual_stream)

      ten_two_output = simulate_head(second_residual_stream, 10, 2)[:, -1, :]
      ten_six_output = simulate_head(second_residual_stream, 10, 6)[:, -1, :]
      ten_ten_output = simulate_head(second_residual_stream, 10, 10)[:, -1, :]

      # constraint three - make the vector perp to the IO direction
      perp_io_loss = F.cosine_similarity(repeated_learned_vector, unembed_io_directions, dim=-1).mean(0).abs()

      silence_loss = backup_loss_metric(ten_two_output, ablated_output = clean_ten_two) + backup_loss_metric(ten_six_output, ablated_output = clean_ten_six) + backup_loss_metric(ten_ten_output, ablated_output = clean_ten_ten) #+ perp_to_9_9_metric(trained_vector)


      similarity_to_other_learned_vector_loss =  0 #F.cosine_similarity(trained_vector, learned_vectors[0], dim=0).abs() + F.cosine_similarity(trained_vector, learned_vectors[7], dim=0).abs()
      if len(learned_vectors) != 0:
        for i in range(len(learned_vectors)):
          similarity_to_other_learned_vector_loss += F.cosine_similarity(trained_vector, learned_vectors[i], dim=0).abs()
        similarity_to_other_learned_vector_loss = similarity_to_other_learned_vector_loss / len(learned_vectors) * 6



      loss = activate_loss + 2 *  silence_loss + perp_io_loss + normalize_metric(trained_vector) + similarity_to_other_learned_vector_loss
      #print(activate_loss.item(), silence_loss.item(), loss.item())
      loss.backward()#retain_graph=True)
      optimizer.step()
  learned_vectors.append(trained_vector)


100%|██████████| 3/3 [01:01<00:00, 20.59s/it]


In [ ]:
# prompt: calculate the cosine similarities between the vectors in learned_vectors

cosine_similarities = torch.zeros((len(learned_vectors), len(learned_vectors)))
for i in range(len(learned_vectors)):
 for j in range(len(learned_vectors)):
        cosine_similarities[i][j] = (F.cosine_similarity(learned_vectors[i], learned_vectors[j], dim=0))

In [ ]:
imshow(cosine_similarities, title = "cos sims between the learned 'backup vectors'", width = 600
       )

In [ ]:
perp_to_9_9_metric(learned_vectors[2])

tensor(0.1617, device='cuda:0', grad_fn=<AbsBackward0>)

In [ ]:
# get length of trained vector
for vec in learned_vectors:
  print(einops.einsum(vec, vec, "d_model, d_model -> "))
print(einops.einsum(nine_nine_output, nine_nine_output, "batch d_model, batch d_model -> batch").mean(0))

tensor(20.9580, device='cuda:0', grad_fn=<ViewBackward0>)
tensor(23.3546, device='cuda:0', grad_fn=<ViewBackward0>)
tensor(11.8660, device='cuda:0', grad_fn=<ViewBackward0>)
tensor(14.1227, device='cuda:0', grad_fn=<ViewBackward0>)
tensor(13.2017, device='cuda:0', grad_fn=<ViewBackward0>)
tensor(26.7330, device='cuda:0', grad_fn=<ViewBackward0>)
tensor(10.6086, device='cuda:0', grad_fn=<ViewBackward0>)
tensor(12.0525, device='cuda:0', grad_fn=<ViewBackward0>)
tensor(18.0147, device='cuda:0', grad_fn=<ViewBackward0>)
tensor(16.7909, device='cuda:0', grad_fn=<ViewBackward0>)
tensor(1723.6044, device='cuda:0')


In [ ]:
torch.set_grad_enabled(False)

## Test the Vector to see if this actually works well!

In [ ]:
model.set_use_attn_result(True)

In [ ]:
for vec in learned_vectors:
  print(perp_to_9_9_metric(vec, output_9_9 = nine_six_output))

tensor(0.0042, device='cuda:0')
tensor(0.0234, device='cuda:0')
tensor(0.0077, device='cuda:0')
tensor(0.0105, device='cuda:0')
tensor(0.0008, device='cuda:0')
tensor(0.0033, device='cuda:0')
tensor(0.0062, device='cuda:0')
tensor(0.0055, device='cuda:0')
tensor(0.0213, device='cuda:0')
tensor(0.0035, device='cuda:0')


In [ ]:
heads_whose_output_to_replace

[(9, 9)]

In [ ]:
repeated_trained_vector = einops.repeat(learned_vectors[2], "d_model -> batch d_model", batch = nine_nine_output.shape[0])

project_stuff_on_heads([(9,9)], repeated_trained_vector, project_only = False, scale_proj = 1, output = "display_logits", freeze_ln=False)

RUH


In [ ]:
heads =  [(9,6)]
model.reset_hooks() # callum library buggy
def return_item(item):
  return item

model.reset_hooks()
patched_logits = act_patch(
    model = model,
    orig_input = clean_tokens,
    new_cache = corrupted_cache,
    patching_nodes = [Node("z", layer = layer, head = head) for layer, head in heads],
    patching_metric = return_item,
    verbose = False,
    apply_metric_to_cache = False
)

nine_six_sample_ablating_cache = act_patch(
    model = model,
    orig_input = clean_tokens,
    new_cache = corrupted_cache,
    patching_nodes = [Node("z", layer = layer, head = head) for layer, head in heads],
    patching_metric = return_item,
    verbose = False,
    apply_metric_to_cache = True
)


model.reset_hooks()
nine_six_sample_ablating_results = act_patch(
    model = model,
    orig_input = clean_tokens,
    new_cache = corrupted_cache,
    patching_nodes = [Node("z", layer = layer, head = head) for layer, head in heads],
    patching_metric = partial(display_corrupted_clean_logits, title = f"Logit Differences When Sample Ablating in Name Mover Heads", comparison = True, logits = patched_logits),
    verbose = False,
    apply_metric_to_cache = True
)

In [ ]:
just_mystery_vector = project_stuff_on_heads(heads_whose_output_to_replace, repeated_trained_vector, project_only = True, scale_proj = 1, output = "get_ldd", freeze_ln=True)
away_mystery_vector = project_stuff_on_heads(heads_whose_output_to_replace, repeated_trained_vector, project_only = False, scale_proj = 1, output = "get_ldd", freeze_ln=True)

RUH
RUH


In [ ]:
just_mystery_vectors = []
away_mystery_vectors = []

for vec in learned_vectors:
  repeated_trained_vector = einops.repeat(vec, "d_model -> batch d_model", batch = nine_nine_output.shape[0])

  just_mystery_vectors.append(project_stuff_on_heads(heads_whose_output_to_replace, repeated_trained_vector, project_only = True, scale_proj = 1, output = "get_ldd", freeze_ln=True))
  away_mystery_vectors.append(project_stuff_on_heads(heads_whose_output_to_replace, repeated_trained_vector, project_only = False, scale_proj = 1, output = "get_ldd", freeze_ln=True))

RUH
RUH
RUH
RUH
RUH
RUH
RUH
RUH
RUH
RUH
RUH
RUH
RUH
RUH
RUH
RUH
RUH
RUH
RUH
RUH


In [ ]:
activate_interventions = {
    "Clean Run" : ca - ca,
    #"Sample Ablation of NMHs" : noise_sample_ablating_results,
    "Sample Ablation of (9.6)" : nine_six_sample_ablating_results,
    "Zero Ablation of NMHs" : first_intervention[0],
    #"Project Only IO Direction (Zero ⊥ IO direction)" : first_intervention[1],
    #"Replace ⊥ IO directions with Corrupted ⊥ IO directions" : first_intervention[2],
    #"Project Away IO Direction (Zero IO direction)" : first_intervention[3],
    #"Replace IO directions with Corrupted IO directions" : first_intervention[4],


    "Project Only (9.6)" : just_9_6,
    "Project Away (9.6)" : away_just_9_6,
    "Project Only (9.9)" : just_9_9,
    "Project Away (9.9)" : away_just_9_9,
    #"Projecting Only (9.6 ∥ 9.9) ∥ IO" : ldd_from_96_99_feature_and_parra_to_io,
    #"Projecting Away (9.6 ∥ 9.9) ∥ IO" : ldd_from_away_96_99_feature_and_parra_to_io,
    #"Project Only (9.6 ∥ 9.9)" : overlap_9_6_9_9,
    #"Project Away (9.6 ∥ 9.9)" : away_overlap_9_6_9_9,
    #"Just the Learned Vector" : just_mystery_vector,
    #"Project Away the Learned Vector" : away_mystery_vector
}

for i, results in enumerate(just_mystery_vectors):
    activate_interventions["just " + str(i)] = results

for i, results in enumerate(away_mystery_vectors):
    activate_interventions["away " + str(i)] = results


In [ ]:
row_labels = [str(i) for i in activate_interventions.keys()]
values = [activate_interventions[key] for key in activate_interventions.keys()]

fig = compare_intervention_ldds_with_sample_ablated(values, row_labels, heads = key_backup_heads , just_logits = True) # + neg_m_heads

In [ ]:
model.set_use_attn_result(False)